In [2]:
from util import *
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from mlxtend.classifier import StackingCVClassifier,StackingClassifier
from sklearn.multiclass import  OneVsOneClassifier,OneVsRestClassifier
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier
from sklearn.preprocessing import StandardScaler,Normalizer
from sklearn.ensemble import VotingClassifier
pd.set_option('display.max_colwidth',1000)

In [3]:
train_all = load_train()
test_all = load_testA()
preprocess_basic_time(train_all)
preprocess_basic_wifi(train_all)
shop_info = load_shop_info()

In [4]:
def get_top_k(pba,k = 3, min_proba = 0):
    _pba = pba.copy()
    def top_k(x,k):
        rs = []
        for _ in range(k):
            ind = np.argmax(x)
            if x[ind] < min_proba:
                continue
            rs.append(ind)
            x[ind] = 0
        return rs
    r = map(lambda x: top_k(x,k), _pba)
    return r

def acc_top_k(candidate, y):
    all_size = len(candidate)
    cor = 0
    for _can,_true in zip(candidate,y):
        if _true in _can:
            cor += 1
    return float(cor) / all_size

def corr_analyse_in_time(x, train, shops_cor_one_day,time=30): # 一天交易多次， 去某个商店在time分钟内可能去哪些商店
    shopids = train[(train.user_id == x[0]) & (train.dayofyear == x[1])][["shop_id","hour_minute"]].values
    for _i,_s in enumerate(shopids):
        for _j,_s2 in enumerate(shopids):
            if _i == _j:
                continue
            else:
                if abs(_s[1] - _s2[1]) > time:
                    continue
                if _s[0] not in shops_cor_one_day:
                    shops_cor_one_day[_s[0]] = {}
                if _s2[0] not in shops_cor_one_day[_s[0]]:
                    shops_cor_one_day[_s[0]][_s2[0]] = 0
                shops_cor_one_day[_s[0]][_s2[0]] += 1
    


def copy_candidate(old_candidate):
    new_candidate = []
    for _nda in old_candidate:
        new_candidate.append(_nda.copy())
    return new_candidate

            
def statistic_candidate(candidate, valid_y = None):
    candi_num = [len(_can) for _can in candidate]
    can_num = np.bincount(candi_num)
    correct = np.zeros((len(can_num),))
    if valid_y is not None:
        for _can, _true in zip(candidate,valid_y):
            if _true in _can:
                correct[len(_can)] += 1
        return can_num, correct.astype(int)
    else:
        return can_num,None  
    
def print_statistic_candidate(candidate, valid_y = None):
    can_num,cor_num = statistic_candidate(candidate, valid_y)
    print "all num",can_num
    if valid_y is not None:
        print "cor num",cor_num
        
def candidate_set(candidate):
    # 在all_rf产生的候选集合中用那一部分商场训练预测
    many = 0
    single = 0
    two = 0
    two_set = set()
    many_set = set()
    for _sample_index,_can in enumerate(candidate):
        if len(_can) ==1:
            single += 1
        else:
            if len(_can) == 2:
                two_set.add(",".join(sorted(list(_can.astype(str)))))
                two +=1
            else:
                many_set.add(",".join(sorted(list(_can.astype(str)))))
                many +=1
    print "many size",many
    print "two size",two
    print "single size",single
    print "two set length",len(two_set)
    print "many set length",len(many_set)
    return two_set,many_set


def forward_search(estimetor, trainx,trainy,fix_trainx,validx,validy,fix_validx):
    choose = []
    best_acc=0
    cc = 0
    while cc != -1:
        cc = -1
        for _i in range(trainx.shape[1]):
            if _i not in choose:
                curr = choose + [_i]
                ptrainx = np.concatenate([trainx[:,curr], fix_trainx],axis=1)
                pvalidx = np.concatenate([validx[:,curr], fix_validx],axis=1)
                estimetor.fit(ptrainx,trainy)
                _acc = acc(estimetor.predict(pvalidx),validy)
                if _acc > best_acc:
                    best_acc = _acc
                    cc = _i
        if cc != -1:
#             print "choose", cc
#             print "best acc", best_acc
            choose.append(cc)
        if best_acc == 1:
            break
    return choose, best_acc

In [6]:
mall_id = "m_4422" # 6587 ,9 21 36 72 77 86 
train = train_all[train_all.mall_id == mall_id]
# label
y = train.shop_id.values
le = LabelEncoder().fit(y)
y = le.transform(y)
#split
_train_index, _valid_index = get_last_one_week_index(train)
valid = train.iloc[_valid_index]
train = train.iloc[_train_index]
#wifi info
df, (train_index, train_use_wifi, train_matrix), (test_index, test_use_wifi, test_matrix) = get_wifi_cache2(mall_id)
train_wifi_all_x = train_matrix[_train_index]
valid_wifi_all_x = train_matrix[_valid_index]
valid_y = y[_valid_index]
train_y = y[_train_index]
train_lonlats = train[["longitude","latitude"]].values
valid_lonlats = valid[["longitude","latitude"]].values
train_wh = train[["weekday","hour"]].values
valid_wh = valid[["weekday","hour"]].values
train_w = train[["weekday"]].values
valid_w = valid[["weekday"]].values
train_h = train[["hour"]].values
valid_h = valid[["hour"]].values

train["dayofyear"] = train.dt.dt.dayofyear
valid["dayofyear"] = valid.dt.dt.dayofyear

indexs = choose_strong_wifi_index(-90,6,train_wifi_all_x)
train_x = np.concatenate([train_wifi_all_x[:,indexs],train_lonlats,train_wh],axis=1)
valid_x = np.concatenate([valid_wifi_all_x[:,indexs],valid_lonlats,valid_wh],axis=1)
lb = LabelBinarizer().fit(y)
train_b_y = lb.transform(train_y)
valid_b_y = lb.transform(valid_y)

train_times = train[["longitude","latitude","is_weekend"]].values
valid_times = valid[["longitude","latitude","is_weekend"]].values

In [8]:
#  多分类
rf_all = RandomForestClassifier(n_estimators=500,n_jobs=-1,class_weight="balanced", random_state=2017,oob_score=True)
_train_all_x = np.concatenate([train_wifi_all_x[:, indexs], train_lonlats, train_wh],axis=1)
valid_all_x = np.concatenate([valid_wifi_all_x[:, indexs], valid_lonlats, valid_wh],axis=1)
_train_y = train_y.copy()
rf_all.fit(_train_all_x,_train_y)
rf_all_pba = rf_all.predict_proba(valid_all_x)
print "valid acc", acc(rf_all.predict(valid_all_x),valid_y)
print "rf oob score", rf_all.oob_score_

valid acc 0.772634318647
rf oob score 0.794818383023


In [104]:
# 候选2 个, 最小概率0.02
candidate = get_top_k(rf_all_pba, 2, 0.02)
candidate = [rf_all.classes_.take(_can) for _can in candidate]
print acc_top_k(candidate, valid_y)
two_set,_= candidate_set(candidate)
print_statistic_candidate(candidate,valid_y)

0.854991634133
many size 0
two size 4333
single size 1046
two set length 326
many set length 0
all num [   0 1046 4333]
cor num [   0 1046 3553]


In [106]:
# 2fenlei
two_dict = {}
two_dict_indexs = {}
ratio = 0.1
for _ind,_s in enumerate(two_set):
    if _ind % 20 == 0:
        print "{}/{}".format(_ind,len(two_set))
    _ss = _s.split(",")
    s1 = int(_ss[0])
    s2 = int(_ss[1])
    
    s1_train = train[train_b_y[:,s1] == 1]
    s2_train = train[train_b_y[:,s2] == 1]
    s1_valid = valid[valid_b_y[:,s1] == 1]
    s2_valid = valid[valid_b_y[:,s2] == 1]

    s1_wifi_all_x = train_wifi_all_x[train_b_y[:,s1] == 1]
    s2_wifi_all_x = train_wifi_all_x[train_b_y[:,s2] == 1]
    sp1 = int(s1_wifi_all_x.shape[0] * ratio)
    sp2 = int(s2_wifi_all_x.shape[0] * ratio)
    s1_indexs = choose_strong_wifi_index(-115,sp1,s1_wifi_all_x)
    s2_indexs = choose_strong_wifi_index(-115,sp2,s2_wifi_all_x)
    _indexs = list(set(s1_indexs).union(set(s2_indexs)))
    _rf = RandomForestClassifier(n_jobs=-1,n_estimators=188,random_state=2017)
    _train_bool_index = (train_b_y[:,s1] == 1) | (train_b_y[:,s2]==1) 
    _valid_bool_index = (valid_b_y[:,s1] == 1) | (valid_b_y[:,s2]==1) 
    ptrain_x = train_wifi_all_x[_train_bool_index][:,_indexs] 
#     pvalid_x = valid_wifi_all_x[_valid_bool_index][:,_indexs] 
    ptrain_y = train_y[_train_bool_index] 
    ptrain_x = np.concatenate([ptrain_x,
                               train_lonlats[_train_bool_index],
                               train_times[_train_bool_index]
                               ],axis=1)
    _rf.fit(ptrain_x,ptrain_y)
    two_dict[_s] = _rf
    two_dict_indexs[_s] = _indexs

0/326
20/326
40/326
60/326
80/326
100/326
120/326
140/326
160/326
180/326
200/326
220/326
240/326
260/326
280/326
300/326
320/326


In [108]:
last_predict,ones,twos,_ = predict_candidate(candidate,valid_y,two_dict,two_dict_indexs,None,None,-1)
two_predict,two_real,two_error_sample_index,two_errors = twos
one_predict,one_real,one_error_sample_index,one_errors = ones
print "final acc", acc(np.asarray(last_predict),valid_y)
print "one", (np.asarray(one_predict) == np.asarray(one_real)).sum(), len(one_predict)
print "two", (np.asarray(two_predict) == np.asarray(two_real)).sum(), len(two_predict)
print "one error", Counter(one_errors)
print "two error", Counter(two_errors)

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
final acc 0.776910206358
one 1046 1046
two 3133 4333
one error Counter()
two error Counter({'68,70': 31, '70,75': 26, '66,74': 26, '44,53': 21, '67,75': 19, '67,72': 18, '36,50': 18, '25,76': 13, '34,67': 11, '23,41': 11, '70,76': 10, '2,64': 9, '66,67': 9, '57,58': 8, '72,75': 8, '65,70': 7, '76,77': 7, '34,65': 6, '34,66': 6, '28,62': 6, '91,92': 6, '65,71': 6, '20,31': 5, '25,65': 5, '11,33': 5, '67,70': 4, '22,42': 4, '25,70': 4, '20,89': 4, '68,75': 3, '41,65': 3, '43,46': 3, '75,76': 3, '37,65': 3, '68,76': 3, '27,50': 3, '65,75': 3, '19,87': 3, '84,85': 2, '51,92': 2, '18,3': 2, '58,6': 2, '38,74': 2, '23,29': 2, '1,10': 2, '38,66': 2, '17,5': 2, '44,92': 2, '55,79': 2, '65,74': 2, '5,57': 2, '34,75': 2, '58,63': 1, '10,32': 1, '79,84': 1, '67,74': 1, '71,76': 1, '31,65': 1, '7,76': 1, '15,19': 1, '79,90': 1, '75,86': 1, '70,77': 1, '70,72': 1, '70,71':

In [107]:
def predict_candidate(candidate, valid_y, two_dict, two_dict_indexes, many_dict, many_dict_indexs, predict_candi_length=-1):
    # 在all_rf产生的候选集合中用那一部分商场训练预测
    if predict_candi_length is None or predict_candi_length < 1:
        predict_candi_length = -1
    last_predict_by_candidates = []
    single_predict = []
    two_predict = []
    many_predict = []
    single_real = []
    two_real = []
    many_real = []
    one_error_sample_index = []
    two_error_sample_index = []
    many_error_sample_index = []
    one_errors = []
    two_errors = []
    many_errors = []
    for _sample_index,_can in enumerate(candidate):
        if _sample_index % 200 ==0:
            print _sample_index
        if len(_can) == 1 and (predict_candi_length == 1 or predict_candi_length == -1):
            last_predict_by_candidates.append(_can[0])
            single_predict.append(_can[0])
            single_real.append(valid_y[_sample_index])
            if _can[0] != [valid_y[_sample_index]]:
                one_errors.append(_can[0])
                one_error_sample_index.append(_sample_index)
        else:
            if len(_can) == 2 and (predict_candi_length == 2 or predict_candi_length == -1):
                s1 = _can[0]
                s2 = _can[1]
                _k = ",".join(sorted(list(_can.astype(str))))
                _indexs = two_dict_indexes[_k]
                _rf = two_dict[_k]
                pvalid_x = valid_wifi_all_x[[_sample_index]][:,_indexs] 
                pvalid_x = np.concatenate([pvalid_x, valid_lonlats[[_sample_index]], valid_times[[_sample_index]]],axis=1)
                _p1 = _rf.predict(pvalid_x)[0]
                last_predict_by_candidates.append(_p1)
                two_predict.append(_p1)
                two_real.append(valid_y[_sample_index])
                if _p1 != valid_y[_sample_index] and (valid_y[_sample_index] == s1 or valid_y[_sample_index] == s2):
                    two_errors.append(",".join(sorted(list(_can.astype(str)))))
                    two_error_sample_index.append(_sample_index)
            elif len(_can) > 2 and (predict_candi_length > 2 or predict_candi_length == -1):
                _s = ",".join(sorted(list(_can.astype(str))))
                _indexs = many_dict_indexs[_s]
                _rf = many_dict[_s]
                pvalid_x = valid_wifi_all_x[[_sample_index]][:,_indexs] 
                pvalid_x = np.concatenate([pvalid_x, valid_lonlats[[_sample_index]]],axis=1)
                _p1 = _rf.predict(pvalid_x)[0]
                last_predict_by_candidates.append(_p1)
                many_predict.append(_p1)
                many_real.append(valid_y[_sample_index])
                if _p1 != valid_y[_sample_index]:
                    many_errors.append(_s)
                    many_error_sample_index.append(_sample_index)
    return last_predict_by_candidates, (single_predict,single_real,one_error_sample_index,one_errors),(two_predict,two_real,two_error_sample_index,two_errors),(many_predict,many_real,many_error_sample_index,many_errors)         


In [74]:
rf_all.classes_

array([ 0,  1, ..., 92, 93])

In [80]:
le.inverse_transform([66])


array(['s_647507'], dtype=object)

In [ ]:
# 2fenlei + 前向搜索
two_dict = {}
two_dict_indexs = {}
for _ind,_s in enumerate(two_set):
    if _ind % 20 == 0:
        print "{}/{}".format(_ind,len(two_set))
    _ss = _s.split(",")
    s1 = int(_ss[0])
    s2 = int(_ss[1])
    
    s1_train = train[train_b_y[:,s1] == 1]
    s2_train = train[train_b_y[:,s2] == 1]
    s1_valid = valid[valid_b_y[:,s1] == 1]
    s2_valid = valid[valid_b_y[:,s2] == 1]

    s1_wifi_all_x = train_wifi_all_x[train_b_y[:,s1] == 1]
    s2_wifi_all_x = train_wifi_all_x[train_b_y[:,s2] == 1]
    s1_indexs = choose_strong_wifi_index(-115,6,s1_wifi_all_x)
    s2_indexs = choose_strong_wifi_index(-115,6,s2_wifi_all_x)
    _indexs = list(set(s1_indexs).union(set(s2_indexs)))
    _rf = RandomForestClassifier(n_jobs=-1,n_estimators=188,random_state=2017)
    _train_bool_index = (train_b_y[:,s1] == 1) | (train_b_y[:,s2]==1) 
    _valid_bool_index = (valid_b_y[:,s1] == 1) | (valid_b_y[:,s2]==1) 
    ptrain_x = train_wifi_all_x[_train_bool_index][:,_indexs] 
    ptrain_y = train_y[_train_bool_index] 
    
    pvalid_x = valid_wifi_all_x[_valid_bool_index][:,_indexs]
    pvalid_y = valid_y[_valid_bool_index]
    
    ptrain_x = np.concatenate([ptrain_x, train_lonlats[_train_bool_index]],axis=1)
    pvalid_x = np.concatenate([pvalid_x, valid_lonlats[_valid_bool_index]], axis=1)
    _rf.fit(ptrain_x,ptrain_y)
    
    p1 = _rf.predict(pvalid_x)
    valid_acc = acc(p1,pvalid_y)
    
    
    if valid_acc < 0.995:
        fi = zip(_rf.feature_importances_, _indexs)
        fi = sorted(fi,key=lambda x:-x[0])
        find_indexs = []
        for _f in fi[:30]:
            if isinstance(_f[1],int):
                find_indexs.append(_f[1])
        choose,best = forward_search(_rf,
                       train_wifi_all_x[_train_bool_index][:,find_indexs],
                       train_y[_train_bool_index],
                       train_lonlats[_train_bool_index],
                       valid_wifi_all_x[_valid_bool_index][:,find_indexs],
                       pvalid_y,
                       valid_lonlats[_valid_bool_index]
                      )
        if best > valid_acc:
            choose = [find_indexs[_c] for _c in choose]
            ptrain_x = np.concatenate([train_wifi_all_x[_train_bool_index][:,choose], 
                               train_lonlats[_train_bool_index],
                              ],axis=1)
            _indexs = choose
           
        
    _rf.fit(ptrain_x,ptrain_y)
    two_dict[_s] = _rf
    two_dict_indexs[_s] = _indexs